In [1]:
import numpy as np
import matplotlib.pyplot as mp
import matplotlib.animation as anim
import scipy.fftpack as fft
import numpy.random as rand
import sklearn.linear_model as skl
import cvxpy as cvx
import cv2 as cv
from itertools import cycle
from lbfgs import fmin_lbfgs
from jupyterthemes import jtplot

In [15]:
jtplot.reset()
mp.switch_backend("Qt5Agg")
mp.style.use("fast")
mp.rcParams["axes.axisbelow"] = True
mp.rcParams["text.usetex"] = False
mp.rcParams["font.family"] = "sans-serif"
mp.rcParams["figure.figsize"] = (5, 5)
mp.rcParams["figure.dpi"] = 100
mp.rcParams["text.latex.preamble"] = [r"\usepackage{physics}"]

In [16]:
def dct2(x):
    return fft.dct(fft.dct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

def idct2(x):
    return fft.idct(fft.idct(x.T, norm='ortho', axis=0).T, norm='ortho', axis=0)

In [19]:
class PulsateSine:
    
    def __init__(self, N, freq, pulse_period):
        self.f = freq
        self.N = N
        self.pulse_period = pulse_period
        self.count = 0
        self.alpha_val = [0, 1]
        self.alpha_idx = 0
        
    def continue_loop(self):
        for i in cycle(self.alpha_val):
            self.alpha_idx = i
            self.count += 1
            yield self.count
            
    def update(self, continue_loop):
        self.im.set_alpha(self.alpha_idx)
        self.alpha_idx += 1
        return self.im
            
    def run(self):
        t = np.linspace(0, 1, self.N)
        Tx, Ty = np.meshgrid(t, t)
        self.A = np.sin(2*np.pi*self.f*(Tx - Ty))
        
        self.fig = mp.figure()
        self.ax = self.fig.add_subplot(111)
        self.im = self.ax.imshow(self.A, alpha=self.alpha_idx, cmap='gray', origin='lower')
        self.ax.grid(0)
        
        ani = anim.FuncAnimation(self.fig, self.update, self.continue_loop, 
                                 interval=1000, repeat_delay=0)
        mp.tight_layout()
        mp.show(block=1)

In [20]:
PulsateSine(100, 4, 0).run()